In [2]:
import os

# Caminho do arquivo
file_path = r'C:\Users\gusta\Documents\ff2\data\minhas_transacoes.db'

# Verificar se o arquivo existe
if os.path.exists(file_path):
    print(f"O arquivo existe: {file_path}")
    # Verificar permissões de leitura e escrita
    can_read = os.access(file_path, os.R_OK)
    can_write = os.access(file_path, os.W_OK)
    print(f"Permissão de leitura: {can_read}")
    print(f"Permissão de escrita: {can_write}")
else:
    print(f"O arquivo não existe: {file_path}")

O arquivo existe: C:\Users\gusta\Documents\ff2\data\minhas_transacoes.db
Permissão de leitura: True
Permissão de escrita: True


In [12]:
import sqlite3
import pandas as pd

# Conectar ao banco de dados SQLite
conn = sqlite3.connect(r'C:\Users\gusta\Documents\ff2\data\minhas_transacoes.db')
consulta = "SELECT * FROM transacoes_ficticias"
# Executar a consulta e ler os dados em um DataFrame
df = pd.read_sql_query(consulta, conn)

In [13]:
df

,Data,Transação,País,Moeda,Valor,Categoria
0,2024-04-13,investment,Kazakhstan,KPW,14.72,Alimentação
1,2024-04-11,shoulder,Svalbard & Jan Mayen Islands,CAD,65.96,Outros
2,2024-04-20,pay,Sudan,GMD,14.52,Saúde
3,2024-05-26,reason,Macao,MAD,68.74,Transporte
4,2024-04-09,my,Mongolia,OMR,61.66,Lazer e Entretenimento
...,...,...,...,...,...,...
95,2024-04-19,look,Kiribati,BSD,64.87,Alimentação
96,2024-05-13,who,Nigeria,NIO,11.75,Outros
97,2024-04-24,network,Samoa,MDL,16.54,Moradia
98,2024-04-10,main,Cayman Islands,XAF,59.95,Outros


In [3]:
import sqlite3

# Conectar ao banco de dados
conn = sqlite3.connect('minhas_transacoes.db')
cursor = conn.cursor()

# Obter todas as tabelas do banco de dados
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tabelas = cursor.fetchall()

# Limpar todas as tabelas
for tabela in tabelas:
    cursor.execute(f"DELETE FROM {tabela[0]}")
    conn.commit()

# Fechar a conexão
conn.close()

In [1]:
from forex_python.converter import CurrencyCodes

In [5]:
c = CurrencyCodes()
c.get_symbol("R$")

In [6]:
def is_valid_currency(currency_code):
        """
        Verifica se o código da moeda ou símbolo é válido.

        Args:
            currency_code (str): Código ou símbolo da moeda a ser verificado.

        Retorna:
            bool: Verdadeiro se o código ou símbolo da moeda for válido, falso caso contrário.
        """
        c = CurrencyCodes()
        # Verifica se é um código de moeda
        if c.get_symbol(currency_code):
            return True
        # Verifica se é um símbolo de moeda e se o símbolo é reconhecido
        return currency_code in c.get_symbols().values()

In [10]:
def is_valid_currency_symbol(symbol):
    # Lista abrangente de símbolos de moeda
    valid_symbols = [
        "R$", "$", "€", "£", "¥", "₹", "₽", "₩", "₺", "₫", "₪", "₭", "₮", "₨", "₣", "₲", "₼", "₦", "₧", "₨", "៛", "₨", "৳"
    ]
    
    # Verifica se o símbolo está na lista de símbolos válidos
    return symbol in valid_symbols

# Exemplos de uso
print(is_valid_currency_symbol("R$"))  # True
print(is_valid_currency_symbol("$"))   # True
print(is_valid_currency_symbol("€"))   # True
print(is_valid_currency_symbol("ABC")) # False



True
True
True
False


In [1]:
pip install faker

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 20.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
import sqlite3
from faker import Faker
import pandas as pd
import random
from datetime import datetime

# Configurações
DB_PATH = r'C:\Users\gusta\Documents\ff2\data\minhas_transacoes.db'  # Substitua pelo caminho do seu banco de dados
NUM_REGISTROS = 100  # Número de registros fictícios que você deseja gerar
MESES = [4, 5]  # Meses desejados (Janeiro e Fevereiro)
ALLOWED_CATEGORIES = [
    "Moradia", "Transporte", "Alimentação", "Saúde", 
    "Lazer e Entretenimento", "Despesas Pessoais", "Outros"
]

# Crie uma instância do Faker
fake = Faker()

def gerar_data_ficticia(ano, meses):
    mes = random.choice(meses)
    dia = random.randint(1, 28)  # Para simplificar, usamos 28 dias para evitar problemas com meses de 30 ou 31 dias
    return datetime(ano, mes, dia).date()

# Gerar dados fictícios
dados_ficticios = []
for _ in range(NUM_REGISTROS):
    dados_ficticios.append({
        "Data": gerar_data_ficticia(2024, MESES),
        "Transação": fake.word(),
        "País": fake.country(),
        "Moeda": fake.currency_code(),
        "Valor": float(round(fake.pydecimal(left_digits=2, right_digits=2, positive=True), 2)),  # Convertendo para float
        "Categoria": random.choice(ALLOWED_CATEGORIES)  # Escolhe uma categoria aleatória
    })

# Converter para DataFrame
df_ficticio = pd.DataFrame(dados_ficticios)

# Salvar no banco de dados SQLite
with sqlite3.connect(DB_PATH) as conn:
    df_ficticio.to_sql("transacoes_ficticias", conn, if_exists="replace", index=False)
